# Summarizing Text with Amazon Reviews

The objective of this project is to build a model that can create relevant summaries for reviews written about fine foods sold on Amazon. This dataset contains above 500,000 reviews, and is hosted on [Kaggle.](https://www.kaggle.com/snap/amazon-fine-food-reviews)

To build our model we will use a two-layered bidirectional RNN with LSTMs on the input data and two layers, each with an LSTM using bahdanau attention on the target data. [Jaemin Cho's tutorial](https://github.com/j-min/tf_tutorial_plus/tree/master/RNN_seq2seq/contrib_seq2seq) for seq2seq was really helpful to get the code in working order because this is my first project with TensorFlow 1.1; some of the functions are very different from 1.0. The architecture for this model is similar to Xin Pan's and Peter Liu's, here's their [GitHub page.](https://github.com/tensorflow/models/tree/master/textsum)

The sections of this project are:
- Inspecting the Data
- Preparing the Data
- Building the Model
- Training the Model
- Making Our Own Summaries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.1.0


## Insepcting the Data

In [2]:
reviews = pd.read_csv("./amazon-fine-food-reviews/Reviews.csv")

In [3]:
reviews.describe()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,568454.000000,568454.000000,568454.00000,568454.000000,5.684540e+05
mean,284227.500000,1.743817,2.22881,4.183199,1.296257e+09
std,164098.679298,7.636513,8.28974,1.310436,4.804331e+07
min,1.000000,0.000000,0.00000,1.000000,9.393408e+08
25%,142114.250000,0.000000,0.00000,4.000000,1.271290e+09
50%,284227.500000,0.000000,1.00000,5.000000,1.311120e+09
75%,426340.750000,2.000000,2.00000,5.000000,1.332720e+09
max,568454.000000,866.000000,923.00000,5.000000,1.351210e+09


In [4]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
Id                        568454 non-null int64
ProductId                 568454 non-null object
UserId                    568454 non-null object
ProfileName               568438 non-null object
HelpfulnessNumerator      568454 non-null int64
HelpfulnessDenominator    568454 non-null int64
Score                     568454 non-null int64
Time                      568454 non-null int64
Summary                   568427 non-null object
Text                      568454 non-null object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [5]:
reviews.shape

(568454, 10)

In [6]:
reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [7]:
# Check for any nulls values# Check 
reviews.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [8]:
reviews.corr()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
Id,1.000000,0.001227,0.000770,0.010706,0.007912
HelpfulnessNumerator,0.001227,1.000000,0.974689,-0.032590,-0.154818
HelpfulnessDenominator,0.000770,0.974689,1.000000,-0.097986,-0.173289
Score,0.010706,-0.032590,-0.097986,1.000000,-0.062760
Time,0.007912,-0.154818,-0.173289,-0.062760,1.000000


In [9]:
# Remove null values and unneeded features
reviews = reviews.dropna()
reviews = reviews.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator',
                        'Score','Time'], 1)
reviews = reviews.reset_index(drop=True)

In [10]:
reviews.head()

,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,"""Delight"" says it all",This is a confection that has been around a fe...
3,Cough Medicine,If you are looking for the secret ingredient i...
4,Great taffy,Great taffy at a great price. There was a wid...


In [11]:
# Inspecting some of the reviews# Inspe 
for i in range(5):
    print("Review #",i+1)
    print(reviews.Summary[i])
    print(reviews.Text[i])
    print()

Review # 1
Good Quality Dog Food
I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.

Review # 2
Not as Advertised
Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".

Review # 3
"Delight" says it all
This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces 

## Preparing the Data

In [12]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [13]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

We will remove the stopwords from the texts because they do not provide much use for training our model. However, we will keep them for our summaries so that they sound more like natural phrases.

In [14]:

# Clean the summaries and texts# Clean t 
clean_summaries = []
for summary in reviews.Summary:
    clean_summaries.append(clean_text(summary, remove_stopwords=False))
print("Summaries are complete.")

clean_texts = []
for text in reviews.Text:
    clean_texts.append(clean_text(text))
print("Texts are complete.")

Summaries are complete.
Texts are complete.


In [15]:
# Inspect the cleaned summaries and texts to ensure they have been cleaned well
for i in range(5):
    print("Clean Review #",i+1)
    print(clean_summaries[i])
    print(clean_texts[i])
    print()

Clean Review # 1
good quality dog food
bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better

Clean Review # 2
not as advertised
product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted sure error vendor intended represent product jumbo

Clean Review # 3
 delight  says it all
confection around centuries light pillowy citrus gelatin nuts case filberts cut tiny squares liberally coated powdered sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story c lewis lion witch wardrobe treat seduces edmund selling brother sisters witch

Clean Review # 4
cough medicine
looking secret ingredient robitussin believe found got addition root beer extract ordered good made cherry soda flavor medicinal

Clean Review # 5
great taffy
great taffy great price wide assortment yummy taffy delivery quick taffy lover deal



In [16]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [17]:
# Find the number of times each word was used and the size of the vocabulary
word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 132884


In [18]:
# Load Conceptnet Numberbatch's (CN) embeddings, similar to GloVe, but probably better 
# (https://github.com/commonsense/conceptnet-numberbatch)
embeddings_index = {}
with open('./amazon-fine-food-reviews/numberbatch-en-17.06.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

Word embeddings: 417195


In [19]:
# Find the number of words that are missing from CN, and are used more than our threshold.
missing_words = 0
threshold = 20

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from CN: 3870
Percent of words that are missing from vocabulary: 2.91%


use a threshold of 20, so that words not in CN can be added to our word_embedding_matrix, but they need to be common enough in the reviews so that the model can understand their meaning.

In [20]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 132884
Number of words we will use: 59595
Percent of words we will use: 44.85%


In [21]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))
print(len(vocab_to_int))

59595
59595


In [22]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [23]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 25679933
Total number of UNKs in headlines: 192245
Percent of words that are UNK: 0.75%


In [24]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [25]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

Summaries:
              counts
count  568411.000000
mean        4.181624
std         2.657872
min         0.000000
25%         2.000000
50%         4.000000
75%         5.000000
max        48.000000

Texts:
              counts
count  568411.000000
mean       41.996835
std        42.520873
min         1.000000
25%        18.000000
50%        29.000000
75%        50.000000
max      2085.000000


In [26]:
# Inspect the length of texts
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))

84.0
115.0
207.0


In [27]:
# Inspect the length of summaries
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99))

8.0
9.0
13.0


In [28]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [29]:
# Sort the summaries and texts by the length of the texts, shortest to longest
# Limit the length of summaries and texts based on the min and max ranges.
# Remove reviews that include too many UNKs

sorted_summaries = []
sorted_texts = []
max_text_length = 84
max_summary_length = 13
min_length = 2
unk_text_limit = 1
unk_summary_limit = 0

for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

425615
425615


## Building the Model

In [30]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

In [31]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [32]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    '''Create the encoding layer'''
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
    # Join outputs since we are using a bidirectional RNN
    enc_output = tf.concat(enc_output,2)
    
    return enc_output, enc_state

In [33]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_summary_length):
    '''Create the training logits'''
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       initial_state,
                                                       output_layer) 

    training_logits, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_logits

In [34]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):
    '''Create the inference logits'''
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        initial_state,
                                                        output_layer)
                
    inference_logits, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_logits

In [42]:
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length, 
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    
    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                     input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  text_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')

    dec_cell = tf.contrib.seq2seq.DynamicAttentionWrapper(dec_cell,
                                                          attn_mech,
                                                          rnn_size)
            
    initial_state = tf.contrib.seq2seq.DynamicAttentionWrapperState(enc_state[0],
                                                                    _zero_state_tensors(rnn_size, 
                                                                                        batch_size, 
                                                                                        tf.float32)) 
    with tf.variable_scope("decode"):
        training_logits = training_decoding_layer(dec_embed_input, 
                                                  summary_length, 
                                                  dec_cell, 
                                                  initial_state,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_summary_length)
    with tf.variable_scope("decode", reuse=True):
        inference_logits = inference_decoding_layer(embeddings,  
                                                    vocab_to_int['<GO>'], 
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell, 
                                                    initial_state, 
                                                    output_layer,
                                                    max_summary_length,
                                                    batch_size)

    return training_logits, inference_logits

In [43]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    '''Use the previous functions to create the training and inference logits'''
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    
    return training_logits, inference_logits

In [44]:
def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [45]:
def get_batches(summaries, texts, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

In [46]:
# Set the Hyperparameters
epochs = 100
batch_size = 64
rnn_size = 256
num_layers = 2
learning_rate = 0.005
keep_probability = 0.75

In [47]:
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits.rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
    
    # Create the weights for sequence_loss
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")

Graph is built.


## Training the Model

Since I am training this model on my MacBook Pro, it would take me days if I used the whole dataset. For this reason, I am only going to use a subset of the data, so that I can train it over night. Normally I use [FloydHub's](https://www.floydhub.com/) services for my GPU needs, but it would take quite a bit of time to upload the dataset and ConceptNet Numberbatch, so I'm not going to bother with that for this project.

I chose not use use the start of the subset because I didn't want to make it too easy for my model. The texts that I am using are closer to the median lengths; I thought this would be more fair.


In [49]:
# Subset the data for training
start = 200000
end = start + 50000
sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print("The shortest text length:", len(sorted_texts_short[0]))
print("The longest text length:",len(sorted_texts_short[-1]))

The shortest text length: 25
The longest text length: 31


In [50]:
# Train the Model
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 3 # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

checkpoint = "best_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    #loader = tf.train.import_meta_graph("./" + checkpoint + '.meta')
    #loader.restore(sess, checkpoint)
    
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts_short) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0

            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break

Epoch   1/100 Batch   20/781 - Loss:  4.769, Seconds: 75.75
Epoch   1/100 Batch   40/781 - Loss:  2.879, Seconds: 75.04
Epoch   1/100 Batch   60/781 - Loss:  2.834, Seconds: 79.77
Epoch   1/100 Batch   80/781 - Loss:  2.736, Seconds: 82.77
Epoch   1/100 Batch  100/781 - Loss:  2.792, Seconds: 75.50
Epoch   1/100 Batch  120/781 - Loss:  2.683, Seconds: 73.61
Epoch   1/100 Batch  140/781 - Loss:  2.578, Seconds: 75.70
Epoch   1/100 Batch  160/781 - Loss:  2.584, Seconds: 78.51
Epoch   1/100 Batch  180/781 - Loss:  2.519, Seconds: 93.39
Epoch   1/100 Batch  200/781 - Loss:  2.598, Seconds: 66.09
Epoch   1/100 Batch  220/781 - Loss:  2.590, Seconds: 77.84
Epoch   1/100 Batch  240/781 - Loss:  2.525, Seconds: 57.16
Average loss for this update: 2.805
New Record!
Epoch   1/100 Batch  260/781 - Loss:  2.357, Seconds: 66.08
Epoch   1/100 Batch  280/781 - Loss:  2.222, Seconds: 80.22
Epoch   1/100 Batch  300/781 - Loss:  2.319, Seconds: 69.62
Epoch   1/100 Batch  320/781 - Loss:  2.383, Seconds

Epoch   4/100 Batch  260/781 - Loss:  1.338, Seconds: 65.21
Epoch   4/100 Batch  280/781 - Loss:  1.220, Seconds: 75.92
Epoch   4/100 Batch  300/781 - Loss:  1.360, Seconds: 67.76
Epoch   4/100 Batch  320/781 - Loss:  1.418, Seconds: 67.31
Epoch   4/100 Batch  340/781 - Loss:  1.647, Seconds: 56.81
Epoch   4/100 Batch  360/781 - Loss:  1.490, Seconds: 78.33
Epoch   4/100 Batch  380/781 - Loss:  1.436, Seconds: 76.43
Epoch   4/100 Batch  400/781 - Loss:  1.300, Seconds: 75.54
Epoch   4/100 Batch  420/781 - Loss:  1.257, Seconds: 60.66
Epoch   4/100 Batch  440/781 - Loss:  1.284, Seconds: 65.60
Epoch   4/100 Batch  460/781 - Loss:  1.456, Seconds: 64.80
Epoch   4/100 Batch  480/781 - Loss:  1.536, Seconds: 64.01
Epoch   4/100 Batch  500/781 - Loss:  1.408, Seconds: 76.65
Average loss for this update: 1.403
New Record!
Epoch   4/100 Batch  520/781 - Loss:  1.425, Seconds: 72.92
Epoch   4/100 Batch  540/781 - Loss:  1.267, Seconds: 58.22
Epoch   4/100 Batch  560/781 - Loss:  1.273, Seconds

Average loss for this update: 1.12
New Record!
Epoch   7/100 Batch  520/781 - Loss:  1.155, Seconds: 70.64
Epoch   7/100 Batch  540/781 - Loss:  1.024, Seconds: 57.44
Epoch   7/100 Batch  560/781 - Loss:  1.008, Seconds: 62.43
Epoch   7/100 Batch  580/781 - Loss:  1.079, Seconds: 61.25
Epoch   7/100 Batch  600/781 - Loss:  1.124, Seconds: 64.03
Epoch   7/100 Batch  620/781 - Loss:  1.296, Seconds: 72.50
Epoch   7/100 Batch  640/781 - Loss:  1.160, Seconds: 78.77
Epoch   7/100 Batch  660/781 - Loss:  1.154, Seconds: 67.28
Epoch   7/100 Batch  680/781 - Loss:  1.013, Seconds: 63.11
Epoch   7/100 Batch  700/781 - Loss:  1.014, Seconds: 84.69
Epoch   7/100 Batch  720/781 - Loss:  1.095, Seconds: 67.94
Epoch   7/100 Batch  740/781 - Loss:  1.381, Seconds: 53.55
Epoch   7/100 Batch  760/781 - Loss:  1.338, Seconds: 58.53
Average loss for this update: 1.142
No Improvement.
Epoch   7/100 Batch  780/781 - Loss:  1.172, Seconds: 67.11
Epoch   8/100 Batch   20/781 - Loss:  1.399, Seconds: 70.72
E

Epoch  10/100 Batch  760/781 - Loss:  1.164, Seconds: 57.54
Average loss for this update: 0.987
No Improvement.
Epoch  10/100 Batch  780/781 - Loss:  1.024, Seconds: 68.45
Epoch  11/100 Batch   20/781 - Loss:  1.211, Seconds: 70.30
Epoch  11/100 Batch   40/781 - Loss:  1.106, Seconds: 71.41
Epoch  11/100 Batch   60/781 - Loss:  1.036, Seconds: 61.47
Epoch  11/100 Batch   80/781 - Loss:  0.991, Seconds: 75.19
Epoch  11/100 Batch  100/781 - Loss:  1.007, Seconds: 66.00
Epoch  11/100 Batch  120/781 - Loss:  0.908, Seconds: 66.05
Epoch  11/100 Batch  140/781 - Loss:  0.881, Seconds: 67.31
Epoch  11/100 Batch  160/781 - Loss:  0.927, Seconds: 76.50
Epoch  11/100 Batch  180/781 - Loss:  0.981, Seconds: 71.01
Epoch  11/100 Batch  200/781 - Loss:  1.010, Seconds: 61.96
Epoch  11/100 Batch  220/781 - Loss:  0.983, Seconds: 65.97
Epoch  11/100 Batch  240/781 - Loss:  0.921, Seconds: 52.23
Average loss for this update: 0.989
No Improvement.
Epoch  11/100 Batch  260/781 - Loss:  0.883, Seconds: 66

Epoch  14/100 Batch  220/781 - Loss:  0.875, Seconds: 65.23
Epoch  14/100 Batch  240/781 - Loss:  0.827, Seconds: 49.71
Average loss for this update: 0.887
No Improvement.
Epoch  14/100 Batch  260/781 - Loss:  0.794, Seconds: 64.51
Epoch  14/100 Batch  280/781 - Loss:  0.710, Seconds: 75.68
Epoch  14/100 Batch  300/781 - Loss:  0.832, Seconds: 64.68
Epoch  14/100 Batch  320/781 - Loss:  0.821, Seconds: 65.42
Epoch  14/100 Batch  340/781 - Loss:  0.960, Seconds: 55.03
Epoch  14/100 Batch  360/781 - Loss:  0.876, Seconds: 75.12
Epoch  14/100 Batch  380/781 - Loss:  0.844, Seconds: 73.92
Epoch  14/100 Batch  400/781 - Loss:  0.801, Seconds: 75.93
Epoch  14/100 Batch  420/781 - Loss:  0.753, Seconds: 59.17
Epoch  14/100 Batch  440/781 - Loss:  0.791, Seconds: 64.52
Epoch  14/100 Batch  460/781 - Loss:  0.830, Seconds: 65.98
Epoch  14/100 Batch  480/781 - Loss:  0.904, Seconds: 61.20
Epoch  14/100 Batch  500/781 - Loss:  0.810, Seconds: 76.25
Average loss for this update: 0.83
New Record!
E

Epoch  17/100 Batch  460/781 - Loss:  0.771, Seconds: 63.47
Epoch  17/100 Batch  480/781 - Loss:  0.832, Seconds: 61.13
Epoch  17/100 Batch  500/781 - Loss:  0.756, Seconds: 76.22
Average loss for this update: 0.767
New Record!
Epoch  17/100 Batch  520/781 - Loss:  0.819, Seconds: 69.03
Epoch  17/100 Batch  540/781 - Loss:  0.726, Seconds: 54.63
Epoch  17/100 Batch  560/781 - Loss:  0.714, Seconds: 59.56
Epoch  17/100 Batch  580/781 - Loss:  0.768, Seconds: 61.95
Epoch  17/100 Batch  600/781 - Loss:  0.758, Seconds: 55.38
Epoch  17/100 Batch  620/781 - Loss:  0.854, Seconds: 71.01
Epoch  17/100 Batch  640/781 - Loss:  0.778, Seconds: 75.13
Epoch  17/100 Batch  660/781 - Loss:  0.817, Seconds: 66.91
Epoch  17/100 Batch  680/781 - Loss:  0.693, Seconds: 60.77
Epoch  17/100 Batch  700/781 - Loss:  0.716, Seconds: 75.46
Epoch  17/100 Batch  720/781 - Loss:  0.771, Seconds: 66.83
Epoch  17/100 Batch  740/781 - Loss:  0.948, Seconds: 50.55
Epoch  17/100 Batch  760/781 - Loss:  0.938, Seconds

Epoch  20/100 Batch  700/781 - Loss:  0.671, Seconds: 74.80
Epoch  20/100 Batch  720/781 - Loss:  0.732, Seconds: 67.44
Epoch  20/100 Batch  740/781 - Loss:  0.896, Seconds: 50.24
Epoch  20/100 Batch  760/781 - Loss:  0.894, Seconds: 56.53
Average loss for this update: 0.743
No Improvement.
Epoch  20/100 Batch  780/781 - Loss:  0.782, Seconds: 65.70
Epoch  21/100 Batch   20/781 - Loss:  0.916, Seconds: 69.26
Epoch  21/100 Batch   40/781 - Loss:  0.816, Seconds: 68.65
Epoch  21/100 Batch   60/781 - Loss:  0.773, Seconds: 59.22
Epoch  21/100 Batch   80/781 - Loss:  0.740, Seconds: 73.30
Epoch  21/100 Batch  100/781 - Loss:  0.755, Seconds: 62.76
Epoch  21/100 Batch  120/781 - Loss:  0.681, Seconds: 64.62
Epoch  21/100 Batch  140/781 - Loss:  0.667, Seconds: 65.11
Epoch  21/100 Batch  160/781 - Loss:  0.711, Seconds: 73.08
Epoch  21/100 Batch  180/781 - Loss:  0.741, Seconds: 68.02
Epoch  21/100 Batch  200/781 - Loss:  0.769, Seconds: 58.81
Epoch  21/100 Batch  220/781 - Loss:  0.734, Sec

Epoch  24/100 Batch  160/781 - Loss:  0.669, Seconds: 76.99
Epoch  24/100 Batch  180/781 - Loss:  0.730, Seconds: 70.80
Epoch  24/100 Batch  200/781 - Loss:  0.718, Seconds: 61.23
Epoch  24/100 Batch  220/781 - Loss:  0.680, Seconds: 64.78
Epoch  24/100 Batch  240/781 - Loss:  0.656, Seconds: 50.62
Average loss for this update: 0.706
No Improvement.
Epoch  24/100 Batch  260/781 - Loss:  0.630, Seconds: 66.49
Epoch  24/100 Batch  280/781 - Loss:  0.569, Seconds: 74.41
Epoch  24/100 Batch  300/781 - Loss:  0.697, Seconds: 65.47
Epoch  24/100 Batch  320/781 - Loss:  0.674, Seconds: 64.93
Epoch  24/100 Batch  340/781 - Loss:  0.761, Seconds: 60.23
Epoch  24/100 Batch  360/781 - Loss:  0.701, Seconds: 80.91
Epoch  24/100 Batch  380/781 - Loss:  0.672, Seconds: 76.72
Epoch  24/100 Batch  400/781 - Loss:  0.656, Seconds: 76.25
Epoch  24/100 Batch  420/781 - Loss:  0.625, Seconds: 70.04
Epoch  24/100 Batch  440/781 - Loss:  0.645, Seconds: 67.13
Epoch  24/100 Batch  460/781 - Loss:  0.669, Sec

Epoch  27/100 Batch  400/781 - Loss:  0.622, Seconds: 81.12
Epoch  27/100 Batch  420/781 - Loss:  0.591, Seconds: 66.74
Epoch  27/100 Batch  440/781 - Loss:  0.622, Seconds: 74.29
Epoch  27/100 Batch  460/781 - Loss:  0.626, Seconds: 76.45
Epoch  27/100 Batch  480/781 - Loss:  0.682, Seconds: 62.53
Epoch  27/100 Batch  500/781 - Loss:  0.614, Seconds: 77.23
Average loss for this update: 0.638
New Record!
Epoch  27/100 Batch  520/781 - Loss:  0.681, Seconds: 77.83
Epoch  27/100 Batch  540/781 - Loss:  0.616, Seconds: 56.65
Epoch  27/100 Batch  560/781 - Loss:  0.607, Seconds: 62.67
Epoch  27/100 Batch  580/781 - Loss:  0.629, Seconds: 66.38
Epoch  27/100 Batch  600/781 - Loss:  0.646, Seconds: 59.05
Epoch  27/100 Batch  620/781 - Loss:  0.707, Seconds: 75.32
Epoch  27/100 Batch  640/781 - Loss:  0.637, Seconds: 80.27
Epoch  27/100 Batch  660/781 - Loss:  0.676, Seconds: 66.91
Epoch  27/100 Batch  680/781 - Loss:  0.577, Seconds: 61.62
Epoch  27/100 Batch  700/781 - Loss:  0.605, Seconds

Epoch  30/100 Batch  640/781 - Loss:  0.610, Seconds: 88.44
Epoch  30/100 Batch  660/781 - Loss:  0.650, Seconds: 75.40
Epoch  30/100 Batch  680/781 - Loss:  0.557, Seconds: 68.34
Epoch  30/100 Batch  700/781 - Loss:  0.565, Seconds: 90.03
Epoch  30/100 Batch  720/781 - Loss:  0.635, Seconds: 74.44
Epoch  30/100 Batch  740/781 - Loss:  0.768, Seconds: 64.49
Epoch  30/100 Batch  760/781 - Loss:  0.749, Seconds: 63.08
Average loss for this update: 0.636
No Improvement.
Epoch  30/100 Batch  780/781 - Loss:  0.669, Seconds: 77.49
Epoch  31/100 Batch   20/781 - Loss:  0.773, Seconds: 77.02
Epoch  31/100 Batch   40/781 - Loss:  0.720, Seconds: 81.47
Epoch  31/100 Batch   60/781 - Loss:  0.662, Seconds: 68.15
Epoch  31/100 Batch   80/781 - Loss:  0.638, Seconds: 89.41
Epoch  31/100 Batch  100/781 - Loss:  0.650, Seconds: 79.90
Epoch  31/100 Batch  120/781 - Loss:  0.595, Seconds: 91.25
Epoch  31/100 Batch  140/781 - Loss:  0.581, Seconds: 74.57
Epoch  31/100 Batch  160/781 - Loss:  0.600, Sec

## Making Our Own Summaries

To see the quality of the summaries that this model can generate, you can either create your own review, or use a review from the dataset. You can set the length of the summary to a fixed value, or use a random value like I have here.

In [52]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [61]:
import pandas as pd

In [62]:
df = pd.read_csv('Verbatims_data2.csv')

In [63]:
reviews_df = df[['REVIEW_TEXT', 'title', 'RATING']].copy()
reviews_df.columns = ['review_text', 'title', 'rating']
reviews_df.head()

,review_text,title,rating
0,Great app,Great,5
1,Great,Great,5
2,Please include a way to clear all notification...,Notifications!!!,3
3,I have been working on a efile for 9 days th...,None,1
4,It now forces you to pay 30 dollars if you wa...,None,1


In [67]:
reviews_df['review_text'][:10]

0                                           Great app 
1                                                Great
2    Please include a way to clear all notification...
3     I have been working on a efile for 9 days  th...
4     It now forces you to pay 30 dollars if you wa...
5        Love it, keeps me organized and simple to use
6    By far my favorite financial app, makes everyt...
7    I used MYOB for years and naturally moved to e...
8     I love the reminders when bills are coming due. 
9    Looks promising but a number of my accounts, t...
Name: review_text, dtype: object

In [86]:
input_sentences = reviews_df['review_text'][:100]
texts = [text_to_seq(sentence) for sentence in input_sentences if sentence.strip() != '']

In [87]:
checkpoint = "./best_model.ckpt"
answer_logits = []

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    for k, text in enumerate(texts):
        #Multiply by batch_size to match the model's input parameters
        answer_logit = sess.run(logits, {input_data: [text]*batch_size, 
                                          summary_length: [np.random.randint(5,8)], 
                                          text_length: [len(text)]*batch_size,
                                          keep_prob: 1.0})[0] 
        answer_logits.append(answer_logit)
    
        # Remove the padding from the tweet
        pad = vocab_to_int["<PAD>"] 

        print('\nOriginal:', input_sentences[k])

        # print('\nText')
        # print('  Word Ids:    {}'.format([i for i in texts[k]]))
        # print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

        # print('Summary')
        # print('  Word Ids:       {}'.format([i for i in answer_logits[k] if i != pad]))
        print('Summary: {}'.format(" ".join([int_to_vocab[i] for i in answer_logit if i != pad])))


INFO:tensorflow:Restoring parameters from ./best_model.ckpt

Original:  Great app 
Summary: great cool cool cool cool cool

Original: Great
Summary: great great great great great great

Original: Please include a way to clear all notifications. I couldn't possibly go through all of them.
Summary: sorry please please which sorry despite

Original:  I have been working on a efile for 9 days  the first time I used this app a few years ago it was great now I'm ready to punch the screen 
Summary: great time time

Original:  It now forces you to pay 30 dollars if you want to e-file because it requires last years return and it won't remeber the info unless you pay for it 
Summary: check the yeast yeast stay

Original: Love it, keeps me organized and simple to use
Summary: tip doggie treat must lids

Original: By far my favorite financial app, makes everything easier and more convenient.
Summary: my favorite products at home how

Original: I used MYOB for years and naturally moved to essential


Original: You guys are great! Thank you and God bless y'all
Summary: very convenient at walmart look at

Original: Easy and convenient. Keep track of all my accounts/money  in one place and I know where my money is going and helps me budget everything
Summary: very impressed

Original:  Super east to use a Monkey can do it lolol got my federal tax return in EXACTLY 1 WEEK THANK U TURBO TAX MOBLE APP LOVE YOU GUYS!!!!! 
Summary: smart smart nutrition everyday early goes

Original: This program is very useful from a planning perspective to an every day use item to help with budgeting and financial control
Summary: the best

Original: Why would you combine two apps when it wasn't fully ready? Mint Bills was flawless! After this all combined both, I'm having all sorts of problems. Cannot link accounts for whatever reason! I even contacted customer service for help and the best they could do was order me to contact my credit card company. Guess what they said when I contacted them? There i


Original:  I use Turbotax every year! This year my taxes were done in less than 10 minutes and thats with my 2 year old running around. Awesome. 
Summary: perfect simple roast coffees

Original: Perfect for my small business bookkeeping and tracking mileage couldn't be made any easier!!
Summary: cet endurance 00 per stay marks

Original: App wouldn't link to my bank account. USELESS
Summary: very authentic

Original:  Got me more in returns than the software I normally use. And really navigated through the marketplace insurance properly which seemed a nightmare before! 
Summary: great smart smart smart smart

Original:  Great app for tax estimate 
Summary: great for making endurance

Original: Mint,makes it solo easy to keep track of my Financial picture. it's a Great tool.
Summary: excellent automatic use 6 6 ounce

Original:  The best out there! This app has all the features to keep you in track with your bills and expenses. Synchronize your money, investments, credit cards automati